<a href="https://colab.research.google.com/github/ioadeer/bi-bycing-ba-2021/blob/main/bicing_2020_2021_clima_rutas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movilidad Sustentable
## Ciencia de Datos e Inteligencia de Negocios


### Configuración

In [1]:
"""
Bicicletas: Recorridos
Clima: PRCP TAVG TEMP[C] HUM[%] PNM[hPa] DD[gr] FF[km/hr]                                                 
2020-2021
Data acquisition & wrangling
"""
import os
import sys
import urllib.request

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime
# from datetime import date, timedelta
import re


datadir = 'datasets/'

In [2]:
# from google.colab import drive
# drive.mount(
#   '/content/drive/', 
#   # force_remount = True
# )
# if drive:
#   datadir = '/content/drive/MyDrive/Academicos/UTN/BI/datasets/'

# Adquisición


In [2]:
insumos = {
  "recorridos-2020" : {
    "remote" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2020.zip',
    "local" : datadir + 'recorridos-realizados-2020.zip',
    "df" : pd.DataFrame(),
  },
  "recorridos-2021" : {
    "remote" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/bicicletas-publicas/recorridos-realizados-2021.zip',   
    "local" : datadir + 'recorridos-realizados-2021.zip',
    "df" : pd.DataFrame(),
  },
  "clima" : {
    "remote" : '',   
    "local" : datadir + 'weather-data-ba.csv',
    "df" : pd.DataFrame(),
  },
  "estaciones" : {
    "remote" : 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte/estaciones-bicicletas-publicas/nuevas-estaciones-bicicletas-publicas.xlsx',   
    "local" : datadir + 'nuevas_estaciones.xlsx',
    "df" : pd.DataFrame(),
  },
  # "rutas" : {
  #   "remote" : '',   
  #   "local" : datadir + 'rutas.csv',
  #   "df" : pd.DataFrame(),
  # },
}

In [3]:
"""
Cargar insumos
"""
def cargar_insumos( insumos ):
    for i in insumos:
        if not os.path.exists( insumos[i]["local"] ):
            print("Descargando:", insumos[i]["remote"])
            urllib.request.urlretrieve(
              insumos[i]["remote"],
              insumos[i]["local"]
            )
        if "zip" in insumos[i]["local"]:
            insumos[i]["df"] = pd.read_csv(
                insumos[i]["local"],
                compression = 'zip',
                encoding = 'utf-8', 
            )
        if "csv" in insumos[i]["local"]:
            insumos[i]["df"] = pd.read_csv(
                insumos[i]["local"],
            )
        if "xlsx" in insumos[i]["local"]:
            insumos[i]["df"] = pd.read_excel(
                insumos[i]["local"],
            )
        print( 'Cargado: ', insumos[i]["local"] )
    return insumos

In [4]:
insumos = cargar_insumos( insumos )

Cargado:  datasets/recorridos-realizados-2020.zip
Cargado:  datasets/recorridos-realizados-2021.zip
Cargado:  datasets/weather-data-ba.csv
Cargado:  datasets/nuevas_estaciones.xlsx


# Subproductos

In [5]:
derivados = {
  "recorridos" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'recorridos.csv',
    "remote" : '',   
  },
  "vueltas" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'vueltas.csv',
    "remote" : '',   
  },
  "estaciones" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'estaciones.csv',
    "remote" : '',   
  },
  "distancias" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'distancias_estaciones.csv',
    "remote" : '',   
  },
  # "rutas" : {
  #   "df"     : pd.DataFrame(),
  #   "local"  : datadir + 'rutas_estaciones.csv',
  #   "remote" : '',   
  # },
  "demanda" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'demanda.csv',
    "remote" : '',   
  },
  # "salidas" : {
  #   "df"     : pd.DataFrame(),
  #   "local"  : datadir + '',
  #   "remote" : '',   
  # },
  # "arribos" : {
  #   "df"     : pd.DataFrame(),
  #   "local"  : datadir + '',
  #   "remote" : '',   
  # },
  "clima" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'clima_datohorario_2020_2021.csv', 
    "remote" : '',
  },
  "pronostico" : {
    "df"     : pd.DataFrame(),
    "local"  : datadir + 'pronostico_2020_2021.csv', 
    "remote" : '',
  },
}

 # Normalizacion


In [6]:
# renombrar y filtrar columnas  2021
encabezados2021 = {
    'ID'                                  : 'id',
    'Duración'                            : 'duracion_recorrido',
    'Estado cerrado'                      : 'estado_cerrado',
    'ID de ciclista'                      : 'id_usuario',
    'Tipo de ciclista'                    : 'cilicsta_tipo',
    'Modelo de bicicleta'                 : 'bicicleta_modelo',   
    'Id de estación de inicio'            : 'id_estacion_origen', 
    'Nombre de estación de inicio'        : 'nombre_estacion_origen',
    'Fecha de inicio'                     : 'fecha_origen_recorrido',     
    'Origen de viaje'                     : 'direccion_estacion_origen',  
#     'lon_estacion_origen'                : 'lon_estacion_origen',        
#     'lat_estacion_origen'                 : 'lat_estacion_origen',
    'Id de estación de fin de viaje'      : 'id_estacion_destino',     
    'Nombre de estación de fin de viaje'  : 'nombre_estacion_destino',
    'Fecha de fin'                        : 'fecha_destino_recorrido',          
#     'direccion_estacion_destino'          : 'direccion_estacion_origen',           
#     'lon_estacion_destino'               : 'lon_estacion_destino',       
#     'lat_estacion_destino'                : 'lat_estacion_destino',
#     'periodo'                             : 'periodo',  
}
insumos["recorridos-2021"]["df"] = insumos["recorridos-2021"]["df"].rename( 
    columns = encabezados2021
)
insumos["recorridos-2021"]["df"]["id"] = insumos["recorridos-2021"]["df"]['id'].astype('Int64')

In [7]:
# Concatenar recorridos 2020:2021
recorridos = [
    insumos["recorridos-2020"]["df"],
    insumos["recorridos-2021"]["df"]
]
derivados["recorridos"]["df"] = pd.concat( recorridos )
del recorridos  

In [8]:
encabezados = {
    'id'                         : 'id',
    'duracion_recorrido'         : 'duracion',
    'estado_cerrado'             : 'estado_cerrado',
    'id_usuario'                 : 'usuario_id',
#     'cilicsta_tipo'            : 'cilicsta_tipo',
#     'bicicleta_modelo'         : 'bicicleta_modelo',   
    'id_estacion_origen'         : 'origen_id', 
    'nombre_estacion_origen'     : 'origen_nombre',
    'fecha_origen_recorrido'     : 'origen_fecha',     
    'direccion_estacion_origen'  : 'origen_direccion',
    'lat_estacion_origen'        : 'origen_lat',  
    'long_estacion_origen'       : 'origen_lon',        
    'id_estacion_destino'        : 'destino_id',     
    'nombre_estacion_destino'    : 'destino_nombre',
    'fecha_destino_recorrido'    : 'destino_fecha',          
    'direccion_estacion_destino' : 'destino_direccion', 
    'lat_estacion_destino'       : 'destino_lat',          
    'long_estacion_destino'      : 'destino_lon',       
#     'periodo'                    : 'periodo',  
}
derivados["recorridos"]["df"] = derivados["recorridos"]["df"].rename( columns = encabezados )
derivados["recorridos"]["df"] = derivados["recorridos"]["df"][ list( encabezados.values() ) ]

In [9]:
derivados["recorridos"]["df"]['destino_nombre'] = derivados["recorridos"]["df"]['destino_nombre'].fillna(
    '999 - Vacio'
)
derivados["recorridos"]["df"]['destino_nombre'] = derivados["recorridos"]["df"]['destino_nombre'].str.replace(
    'Balboa Definitivo', 
    '666 - Balboa Definitivo', 
    regex = False
)

In [10]:
"""
Separar las columnas nombre de estacion de origen y nombre de estacion de destino 
Tienen la info correcta del id de las estaciones.
Divido la columna Nombre de estacion de fin de viaje porque contiene dos campos 
el Id y el nombre de la estacion de inicio de viaje.
"""
derivados["recorridos"]["df"][
    ['origen_id','origen_nombre']
] = derivados["recorridos"]["df"]['origen_nombre'].str.split('-', expand = True )
derivados["recorridos"]["df"]['origen_id'] = derivados["recorridos"]["df"]['origen_id'].str.strip(' ')
derivados["recorridos"]["df"]['origen_nombre'] = derivados["recorridos"]["df"]['origen_nombre'].str.strip(' ')

derivados["recorridos"]["df"][
    ['destino_id','destino_nombre']
] = derivados["recorridos"]["df"]['destino_nombre'].str.split('-', expand = True )
derivados["recorridos"]["df"]['destino_id'] = derivados["recorridos"]["df"]['destino_id'].str.strip(' ')
derivados["recorridos"]["df"]['destino_nombre'] = derivados["recorridos"]["df"]['destino_nombre'].str.strip(' ')

In [11]:
derivados["recorridos"]["df"]['origen_id'] = derivados["recorridos"]["df"]["origen_id"].str.lstrip('0').astype(int)
derivados["recorridos"]["df"]['destino_id'] = derivados["recorridos"]["df"]["destino_id"].str.lstrip('0').astype(int)
derivados["recorridos"]["df"]["origen_nombre"]  = derivados["recorridos"]["df"]["origen_nombre"].str.upper()
derivados["recorridos"]["df"]["destino_nombre"]  = derivados["recorridos"]["df"]["destino_nombre"].str.upper()
derivados["recorridos"]["df"]["origen_direccion"]  = derivados["recorridos"]["df"]["origen_direccion"].str.upper()
derivados["recorridos"]["df"]["destino_direccion"]  = derivados["recorridos"]["df"]["destino_direccion"].str.upper()

## RESAMPLING (REMUESTREO)

### Downsampling frequencies 

Sample size = minutes

In [12]:
formato = '%Y%m%d %H:%M:%S'
derivados["recorridos"]["df"]['origen_datetime'] = pd.to_datetime(
    derivados["recorridos"]["df"]['origen_fecha'], 
    format = formato
)
derivados["recorridos"]["df"]['fecha'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%Y-%m-%d')
derivados["recorridos"]["df"]['wdia'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%w').astype(str).astype(int)
derivados["recorridos"]["df"].eval( 'dhabil = wdia > 4', inplace = True )
derivados["recorridos"]["df"]['hora'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%H').astype(str).astype(int)

# derivados["recorridos"]["df"]['nmes'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%m').astype(str).astype(int)
# derivados["recorridos"]["df"]['mes']  = bicicletas["recorridos_df"]['start_datetime'].dt.strftime('%b').str.lower()
# derivados["recorridos"]["df"]['ndia'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%j').astype(str).astype(int)
# derivados["recorridos"]["df"]['hora'] = derivados["recorridos"]["df"]['hora'].astype(str).astype(int)
# derivados["recorridos"]["df"]['mins'] = derivados["recorridos"]["df"]['origen_datetime'].dt.strftime('%M')

## SUBPRODUCTOS / CLUSTERING

### GRUPING & RESHAIPING
#### DEMANDA HORARIA

In [ ]:
# https://pandas.pydata.org/docs/user_guide/reshaping.html
# demanda_horaria.pivot( index = "origen_id", columns="hora", values="salidas")
# demanda_horaria.stack()
# https://stackoverflow.com/questions/28651079/pandas-unstack-problems-valueerror-index-contains-duplicate-entries-cannot-re
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

demanda_horaria = derivados["recorridos"]["df"]
# Fitro x dia
# demanda_horaria = demanda_horaria[demanda_horaria["fecha"] == '2020-01-01' ]
demanda_horaria = demanda_horaria.groupby(
    ['origen_nombre', 'origen_id','hora',]
).agg(
  salidas = ( 'hora',  'count'),
)
# .reset_index()
# df.set_index(['id', 'date', 'location'], append=True)
demanda_horaria = demanda_horaria.pivot_table(
    index = ['origen_nombre',  'origen_id'],
    columns ="hora",
    values = "salidas",
    # aggfunc ='sum',
).sort_values('origen_id')
demanda_horaria

### SUMINISTRO HORARIO




In [ ]:
suministro_horario = derivados["recorridos"]["df"].groupby(
    [ 'destino_nombre',  'destino_id', 'hora' ]
).agg(
  salidas = ( 'hora',  'count'),
)
suministro_horario = suministro_horario.pivot_table(
    index = ['destino_nombre',  'destino_id'],
    columns = "hora",
    values = "salidas",
    # aggfunc = 'sum',
).sort_values('destino_id')
suministro_horario

In [ ]:
demanda_diaria = derivados["recorridos"]["df"].groupby(
    ['origen_nombre', 'origen_id','wdia']
).agg(
  salidas = ( 'wdia',  'count'),
)
demanda_diaria = demanda_diaria.pivot_table(
    index   = ['origen_nombre',  'origen_id'],
    columns = "wdia",
    values  = "salidas",
    # aggfunc = 'count',
).sort_values('origen_id')
demanda_diaria 

## SALIDAS Y ARRIBOS

In [ ]:
salidas = derivados["recorridos"]["df"].groupby(
    ['origen_id', 'origen_direccion','origen_nombre','origen_lat','origen_lon' ]
).agg(
  salidas = pd.NamedAgg( column = "origen_id", aggfunc = "count" ),
).reset_index()

# arribos = pd.DataFrame()
arribos = derivados["recorridos"]["df"].groupby(
    ['destino_id', 'destino_direccion','destino_nombre','destino_lat','destino_lon']
).agg(
  arribos = pd.NamedAgg( column = "destino_id", aggfunc = "count" ),
).reset_index()

# del derivados["estaciones"]["df"] 
derivados["estaciones"]["df"] = pd.merge(
    left     = salidas,
    right    = arribos,
    how      = 'left',
    left_on  = ['origen_id','origen_nombre'],
    right_on = ['destino_id','destino_nombre']
)
del [ salidas , arribos ]
orden_filtro = {
    'origen_id'        :'id',
    'origen_direccion' :'direccion',
    'origen_nombre'    :'nombre',
    'origen_lon'       :'lon',
    'origen_lat'       :'lat',
    'salidas'          :'salidas',
    'arribos'          :'arribos',
#     'destino_id',
#     'destino_direccion',
#     'destino_nombre',
#     'destino_lon',
#     'destino_lat',
}
derivados["estaciones"]["df"] = derivados["estaciones"]["df"].rename( columns = orden_filtro )
derivados["estaciones"]["df"] = derivados["estaciones"]["df"][ list( orden_filtro.values() ) ]
derivados["estaciones"]["df"] = derivados["estaciones"]["df"].sort_values(
  by = ['salidas'], ascending = [False]
)

In [ ]:
derivados["estaciones"]["df"]

In [ ]:
# GRABAR CSV ESTACIONES
# derivados["estaciones"]["df"].to_csv(
#   derivados["estaciones"]["local"],
#   index = False,
# )

## FEATURE DISCOVERY

### duracion_cat (catergories)

In [ ]:
# DURACION EN MINUTOS 
derivados["recorridos"]["df"].eval('duracion_mins = duracion / 60', inplace = True)
derivados["recorridos"]["df"]["duracion_mins"] = derivados["recorridos"]["df"]["duracion_mins"].astype(int)
# MINUTOS TO CATEGORIES
# labels = [ "{0}-{1}".format(i, i + 15) for i in range(0, 265, 15) ]
dur_cortes    = [ 0,     5,   15,  30,    45,  60,   90,  120,  240, 320, 60000 ]
dur_etiquetas = [ 'XXS', 'XS', 'S', 'SM', 'M', 'ML', 'L', 'XL', 'XXL', 'OS' ]
derivados["recorridos"]["df"]["duracion_cat"] = pd.cut( 
    derivados["recorridos"]["df"].duracion_mins,
    dur_cortes,
    include_lowest = True,
    labels = dur_etiquetas
   # right = False,
)
derivados["recorridos"]["df"]['duracion_cat'].unique()

In [ ]:
# derivados["recorridos"]["df"][derivados["recorridos"]["df"]['duracion_cat'] == 'M' ]

### TIPO DE RECORRIDO: VUELTA / IDA

In [ ]:
derivados["recorridos"]["df"]['vuelta'] = np.where( 
  derivados["recorridos"]["df"]['origen_id'] == derivados["recorridos"]["df"]['destino_id'], 1, 0
)

In [ ]:
# FILTRAR VUELTAs y duracion_cat
vueltas = derivados["recorridos"]["df"][ derivados["recorridos"]["df"]['vuelta'] == 1]
vueltas[ vueltas['duracion_cat'] == 'OS'  ].sort_values("duracion_mins", ascending = False)


### DISTANCIAS


In [ ]:
# !pip install openrouteservice

In [ ]:
estaciones = [ x for x in derivados["estaciones"]["df"].set_index('id').sort_values('id').iterrows() ]
rutas = {}
nid = 0
for o in estaciones:
    origen_id = int(o[0])
    # https://ask.openrouteservice.org/t/could-not-find-point-within-a-radius-of-350-0-meters/2668/16
    origen_coords = ( o[1]['lon'], o[1]['lat'] ) # ORS takes lon,lat 
    for d in estaciones:
        destino_id = int(d[0])
        destino_coords = ( d[1]['lon'], d[1]['lat'] ) # ibidem
        ruta_id = str( origen_id ) + 'to' + str( destino_id )
        vuelta_id = str( destino_id) + 'to' + str( origen_id )
        ruta = {
          "id" : ruta_id,
          "nid" : nid,
          "origen_id": origen_id,
          "destino_id": destino_id,
          "coords" : ( origen_coords, destino_coords ),
          "distancia" : 0,
          "duracion_estimada" : 0,
          "consultar" : ( origen_id != destino_id ),
          "complemetaria" : vuelta_id,
          "preexiste" : (ruta_id in rutas or vuelta_id in rutas),
        }
    nid+=1
    rutas[ruta_id] = ruta
print( 'rutas:', len( rutas ) )

# del rutas_calcular
rutas_calcular = {}
chunk = 0
cuenta_rutas = 0
for r in rutas:
    ruta = rutas[r]
    if ruta['consultar'] and not ruta["preexiste"]:
        if( (cuenta_rutas % 2000) == 0 ): # due to ors dayly cuota
            chunk = chunk + 1  
        ruta["chunk"] = chunk
        rutas_calcular[r] = ruta
        cuenta_rutas = cuenta_rutas + 1
print( 'rutas a calular:', len( rutas_calcular ) )



In [ ]:
from time import sleep
import openrouteservice as ors

ors_client = ors.Client(
    key = '5b3ce3597851110001cf624827a63bd0239f438caa1f7504352a9fc8'
) 
# del rutas_calculadas
rutas_calculadas = pd.DataFrame()
cuenta_calculadas = 1
for rc in rutas_calcular:
    ruta = rutas[rc]
    if ruta['chunk'] == 1: # CAMBIAR MAÑANA
        if ( ( cuenta_calculadas % 40) == 0): # due to ors minute cuota
            print( 'durmiendo', cuenta_calculadas )
            print( 'rutas calculadas:', rutas_calculadas.size )
            rutas_calculadas
            sleep(60)
        ruta['directions'] = ors_client.directions(
          ruta['coords'],
          profile = 'cycling-regular',
        )
        cuenta_calculadas = cuenta_calculadas + 1
        rutas_calculadas = rutas_calculadas.append(
           ruta,
           ignore_index = True
        )
print( 'rutas calculadas:', rutas_calculadas.size )
# rutas_df = pd.concat(  [ insumos["rutas"]["df"] , rutas_df ] )

rutas_calculadas.to_csv(
  datadir + 'rutas.csv',
  index = False,
)
# rutas_df.reset_index()

In [ ]:
rutas_calculadas

In [ ]:
rutas_df = pd.read_csv(
  datadir + 'rutas.csv',
)
rutas_df

In [ ]:
# GEOPY

# import geopy

# # DF DISTANCIAS
# distancias = pd.DataFrame()
# estaciones = [ x for x in derivados["estaciones"]["df"].set_index('id').sort_values('id').iterrows() ]
# for o in estaciones:
#   print(cuenta_calls)
#   origen_id = int( o[0] )
#   origen_geocoords = ( o[1]['lat'], o[1]['lon'] )
#   # distancias[origen_id] = {}
#   for d in estaciones:
#     destino_id = int( d[0] )
#     destino_geocoords = ( d[1]['lat'], d[1]['lon'] ) 
#     fila = {
#       "id"  : str(origen_id) + 'to' + str(destino_id),
#       "origen_id": origen_id,
#       "destino_id": destino_id,
#       "distancia" : 0,
#     }
#     if (
#         destino_id != origen_id
#     ): 
#       geocoords = ( origen_coords, destino_coords )
#       distancia = round( 
#         geopy.distance.vincenty( origen_geocoords, destino_geocoords ).km, 
#         2
#       ) 
#       fila["distancia"] = distancia
#     distancias = distancias.append( fila, ignore_index = True )
# distancias['origen_id'] = distancias['origen_id'].astype(int)
# distancias
# distancias.to_csv(
#   datadir + 'distancias.csv',
#   index = False,
# )

In [ ]:
distancias

### Data aligment / Pairing

In [ ]:
# derivados["recorridos"]["df"]["distancia"] = 0
derivados["recorridos"]["df"] = pd.merge(
    left     = derivados["recorridos"]["df"],
    right    = distancias,
    how      = 'left', 
    left_on  = ['origen_id','destino_id'], 
    right_on = ['origen_id','destino_id']
)

In [ ]:
derivados["recorridos"]["df"]["distancia"].max()

In [ ]:
# derivados["recorridos"]["df"]["distnacia"].unique()
dis_cortes    = [ 0,    2,   4,    6,   8,    10,   12,  14 ]
dis_etiquetas = [ 'XS', 'S', 'SM', 'M', 'ML', 'L', 'XL']
derivados["recorridos"]["df"]["distancia_cat"] = pd.cut( 
    derivados["recorridos"]["df"].distancia,
    dis_cortes,
    # include_lowest = True,
    labels = dis_etiquetas,
    ordered = True,
   # right = False,
)
derivados["recorridos"]["df"]['distancia_cat'].unique()

In [ ]:
derivados["recorridos"]["df"][derivados["recorridos"]["df"]["vuelta"] == 0 ]

### VELOCIDAD

In [ ]:
derivados["recorridos"]["df"].eval( 'velocidad_kmm = distancia / duracion_mins', inplace = True )
derivados["recorridos"]["df"][ derivados["recorridos"]["df"]["distancia_cat"] == "XL" ]

In [ ]:
column = derivados["recorridos"]["df"][ derivados["recorridos"]["df"]['velocidad_kmm'] != np.inf ]["velocidad_kmm"].dropna()
max_value = column.max()
mean_value = column.mean()
print(max_value)
print(mean_value)
derivados["recorridos"]["df"]["velocidad_kmm"].mean()

In [ ]:
vel_cortes    = [  0,      0.05,     0.1,    0.15,    0.2,     0.25,     0.3,         0.5,        0.7,      1 ]
vel_etiquetas = [ 'slento','lento','normal','nrapido','rapido','xrapido','xxrapido', 'xxxrapido', 'mrapido'   ]
derivados["recorridos"]["df"]["velocidad_cat"] = pd.cut( 
    derivados["recorridos"]["df"].velocidad_kmm,
    vel_cortes,
    include_lowest = True,
    labels = vel_etiquetas,
    ordered = True,
    # right = False,
)
derivados["recorridos"]["df"]['velocidad_cat'].unique()

In [ ]:
derivados["recorridos"]["df"][derivados["recorridos"]["df"]['velocidad_cat'] == 'slento']


### CLIMA
https://www.smn.gob.ar/descarga-de-datos
https://www.smn.gob.ar/datos-abiertos-smn


### Regular sequences: pd.date_range()

Regular date sequences 

In [ ]:
# Lista de dias a obtener
sdate = date(2020,1,1)  # start date
today = datetime.date.today()
yesterday = today - datetime.timedelta( days = 1 )
dias = pd.date_range(
  sdate, 
  yesterday-timedelta( days  1 ),
  freq='d'
)

### Observaciones Horarias
```
$ curl -v --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt 2>&1 | grep AERO
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt | head -n 1
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt | grep AEROPARQUE

```

In [ ]:
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario20210518.txt | grep AEROPARQUE

#### Pronostico
```
curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt | head -n 52
curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt | head -n 18
curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt | head -n 18 | tail -n 8 | awk '/1/ -F ":" {print $1,$2,$7}'
```

In [ ]:
!curl --silent https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=pron5d/pron20210101.txt | head -n 18 | tail -n 8 | awk '/1/ -F ":" {print $1,$2,$7}'

In [ ]:
""" Obtener datos horarios para cada dia  """
dh_formato = '%d%m%Y%H'
prefijo = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=observaciones/datohorario'
sufijo = '.txt'
for i, d in enumerate(dias):
  dstr = d.strftime('%Y%m%d') 
  dato_horario = prefijo + dstr + sufijo
  req = urllib.request.Request( dato_horario )
  response = urllib.request.urlopen( req )
  lines = response.readlines()
  fecha = d.strftime('%Y-%m-%d')
  print(i, 'procesando', dato_horario)
  # encabezados = lines[0].decode("utf-8").split()  
  # ['FECHA', 'HORA', 'TEMP', 'HUM', 'PNM', 'DD', 'FF', 'NOMBRE']
  # [b'[HOA]', b'[\xbaC]', b'[%]', b'[hPa]', b'[gr]', b'[km/hr]']
  # fstl = lines[2].decode("utf-8").split()  
  # print( encabezados )
  # print( fstl )
  
  for l in lines:
    if b'AEROPARQUE' in l:
      row =  l.decode("utf-8").split()
      # dh = str(row[0]) + str(row[1])
      # dh_dt = pd.to_datetime(
      #   dh, 
      #   format = dh_formato
      # ) 
      dh_dic = {
        # 'DH_DT'   : dh_dt,
        'FECHA'   : fecha,
        'HORA'    : row[1],
        'TEMP'    : row[2], 
        'HUM'     : row[3], 
        'PNM'     : row[4], 
        'DD'      : row[5], 
        'FF'      : row[6], 
        # 'NOMBRE'  : row[7]
      }
      derivados["clima"]["df"] = derivados["clima"]["df"].append(
        dh_dic,
        ignore_index=True
      )

In [ ]:
# CARGAR CLIMA
derivados["clima"]["df"] = pd.read_csv(
  derivados["clima"]["local"],
)
derivados["clima"]["df"].info()

In [ ]:
# formato = '%Y-%m-%d'
# derivados["clima"]["df"]['DATE'] = pd.to_datetime(
#   derivados["clima"]["df"]['DATE'], 
#   format = formato
# )
# # derivados["clima"]["df"]['TAVG'].unique()
# derivados["clima"]["df"]['TEMP'].unique()

In [ ]:
derivados["recorridos"]["df"] = pd.merge(
    left     = derivados["recorridos"]["df"],
    right    = derivados["clima"]["df"],
    how      = 'left', 
    left_on  = ['fecha','hora'], 
    right_on = ['FECHA','HORA']
)
derivados["recorridos"]["df"]

In [ ]:
derivados["recorridos"]["df"] = pd.merge(
    left     = derivados["recorridos"]["df"],
    right    = insumos["clima"]["df"],
    how      = 'left', 
    left_on  = 'fecha', 
    right_on = 'DATE'
)

In [ ]:
orden = [
#  'id',
#  'estado_cerrado',
  'usuario_id',
  'origen_id',
  'origen_nombre',
  # 'origen_fecha',
  # 'origen_direccion',
  # 'origen_lon',
  # 'origen_lat',
  'destino_id',
  'destino_nombre',
#  'destino_fecha',
#  'destino_direccion',
#  'destino_lon',
#  'destino_lat',
  'vuelta', 
  'duracion',
  'duracion_mins',
  'duracion_cat',
  'distancia',
  'distancia_cat',
  'velocidad_kmm',
  'velocidad_cat',
  'origen_datetime',
  'fecha',
  'wdia',
  'hora',
  'dhabil',
  'DD',
  'FF',
  'HUM',
  'PNM',
  'TEMP',
  'PRCP',
  'TAVG',
#  'FECHA',
#  'HORA',
#  STATION	NAME	LATITUDE	lonITUDE	ELEVATION	DATE
#  PRCP_ATTRIBUTES
#  TAVG_ATTRIBUTES	TMAX	TMAX_ATTRIBUTES	TMIN	TMIN_ATTRIBUTES                                                
]
derivados["recorridos"]["df"] = derivados["recorridos"]["df"][ orden ]
derivados["recorridos"]["df"]

In [ ]:
# recorridos_clima 
derivados["recorridos"]["df"].to_csv(
  derivados["recorridos"]["local"],
  index = False,
)

In [ ]:
derivados["recorridos"]["df"].info()

In [ ]:
# GRUPING USERS
usurios = derivados["recorridos"]["df"].groupby(
    ['usuario_id']
).agg(
  salidas = ( 'destino_id',  'count'),
  vueltas = ( 'vuelta',  'sum'),
  velocidad = ( 'velocidad_kmm',  'max'),
  distancia = ( 'distancia',  'max')
)
# usurios = usurios.pivot_table(
#     index   = [ 'usuario_id'],
#     columns = "salidas",
#     values  = "salidas",
#     # aggfunc = 'count',
# ).sort_values('origen_id')
# demanda_diaria # comprobar

In [ ]:
usurios = usurios[ ~usurios.isin([np.nan, np.inf, -np.inf]).any(1) ]
# usurios
usurios.sort_values( 'velocidad' , ascending = False).head(30)

# Analisis Exploratorio de Datos Espaciales



In [ ]:
# demanda_viz = derivados["recorridos"]["df"]
# # Fitro x dia

# demanda_viz = demanda_viz[ demanda_viz["fecha"] == '2020-01-01' ]
# demanda_viz

## Exploratory Spatial Data Analysis (ESDA)

https://alcidanalytics.com/p/geographic-heatmap-in-python

https://alysivji.github.io/getting-started-with-folium.html

https://medium.com/analytics-vidhya/measure-driving-distance-time-and-plot-routes-between-two-geographical-locations-using-python-39995dfea7e

In [13]:
demanda_viz = derivados["recorridos"]["df"]
# Fitro x dia
fecha_viz = '2020-03-01'
demanda_viz = demanda_viz[ demanda_viz["fecha"] == fecha_viz ]
demanda_viz = demanda_viz.groupby(
    ['origen_nombre', 'origen_id', 'hora', 'origen_lon','origen_lat']
).agg(
  salidas = ( 'hora',  'count' ),
  nombre = ( 'origen_nombre',  'first' ),
  id = ( 'origen_id',  'first' ),
  lon = ( 'origen_lon',  'first' ),
  lat= ( 'origen_lat',  'first' ),
)
# ).reset_index()
# df.set_index(['id', 'date', 'location'], append=True)
demanda_viz = demanda_viz.pivot_table(
    index = ['nombre',  'id', 'lon','lat'],
    columns = "hora",
    values = "salidas",
    fill_value = 0,
    # aggfunc = 'sum',
).sort_values('id')


7

In [ ]:
# import folium
# from folium import plugins
# # import pandas as pd
# # import matplotlib.pyplot as plt
# import seaborn as sns

# %matplotlib inline




In [14]:
#... where m is my map object. And 5 is the time (seconds) to render the map.
# https://stackoverflow.com/questions/53565979/export-a-folium-map-as-a-png
# https://blog.ouseful.info/2015/12/15/grabbing-screenshots-of-folium-produced-choropleth-leaflet-maps-using-selenium/
# from time import sleep
import io
from PIL import Image

# import the folium library
# !pip install folium selenium
import folium 
from folium.features import DivIcon
import selenium

# from selenium.webdriver import Firefox

# driver = Firefox()
# driver.set_window_size(1920, 1080)  # choose a resolution




import math
def mapear(x): 
    return int( (x / 10 ) * 255)

# initialize the map and store it in a m object
# m = folium.Map(location=[40, -95], zoom_start=4)



In [17]:
# del m
m = ''
for h in demanda_viz.columns.to_list():
    del m
    m = folium.Map(
#         [ -34.606499,-58.381094 ], # obelisco
        [-34.606739, -58.435539 ], # parque centenario
        zoom_start = 13,
#         tiles='stamentoner'
    )
    folium.map.Marker(
        [ -34.559920, -58.372211 ],
        icon = DivIcon(
            icon_size=(600,400),
            icon_anchor=(0,0),
            html='<div style="font-size: 20pt;font-family: monospace;">' \
            + fecha_viz + ' HORA:' + "{:02d}".format( h ) \
            + '</div>',
        )
    ).add_to(m)
    demandas = demanda_viz[h]
    estaciones = demanda_viz[h].reset_index()
    for e in estaciones.iterrows():
        estacion = e[1]
        folium.CircleMarker(
            [ estacion['lat'], estacion['lon'] ],
            radius = estacion[h] * 5, 
            popup = "{0}\:{1}\demanda:{2}".format(
                estacion['id'],
                estacion['nombre'],
                str(estacion[h]),
            ),
            fill_color = "#{0:02x}{1:02x}{2:02x}".format(
                mapear( int(estacion[h]) ),
                255 - mapear( int(estacion[h]) ) ,
                255 - mapear( int(estacion[h]) )
            )
        ).add_to(m)
    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save( 'ESDA/' + fecha_viz + '_' + "{:02d}".format( h ) + '_' + 'map.png' )

In [19]:
# !ffmpeg -pattern_type glob -i './ESDA/*.png' -vf zoompan=z=1:d=4:s=1366x683:fps=5,framerate=25 -vcodec libx264 -acodec aac ESDA.mp4
!ffmpeg -pattern_type glob -i './ESDA/*.png' -vf zoompan=z=1:d=4:s=1366x683:fps=5,framerate=25 -vcodec libx264 -an ESDA.mp4

ffmpeg version n4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.1.0 (GCC)
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxml2 --enable-libxvid --e

## reserva

In [43]:
# # !pip install ffmpeg-python
# import ffmpeg
# # stream = ffmpeg.input('input.mp4')
# # stream = ffmpeg.hflip(stream)
# # stream = ffmpeg.output(stream, 'output.mp4')
# # ffmpeg.run(stream)
# del stream
# stream = ffmpeg.input( 'ESDA/*.png', pattern_type = 'glob', framerate = 24 )
# stream = ffmpeg.output( stream, 'ESDA.mp4')
# ffmpeg.run(stream)


(None, None)

In [ ]:
# import math
# def mapear(x): 
#   return int( (x / 30295 ) * 255) 
# m = folium.Map(
#   [ -34.606499,-58.381094 ],
#   zoom_start = 14
# )
# #  mark each station as a point
# for e in derivados["estaciones"]["df"].iterrows():
#   folium.CircleMarker(
#       [ estacion['lat'], estacion['lon'] ],
#       radius= estacion['salidas'] * 0.002, 
#       popup = "{0}\nsalidas:{1}\narribos:{2}".format(
#           estacion['nombre'],
#           str(estacion['salidas']),
#           str(estacion['arribos'])
#       ),
#       fill_color = "#{0:02x}{1:02x}{2:02x}".format(
#           mapear( int(estacion['salidas']) ),
#           255 - mapear( int(estacion['salidas']) ) ,
#           255 - mapear( int(estacion['salidas']) )
#       )
#   ).add_to(m)
# m

In [ ]:
# # convert to (n, 2) nd-array format for heatmap
# # stationArr = derivados["estaciones"]["df"][['lat', 'lon']].as_matrix()
# # https://stackoverflow.com/a/63293803
# stationArr = derivados["estaciones"]["df"][['salidas']].values

# # plot heatmap
# m.add_children(plugins.HeatMap(stationArr, radius=15))
# m

In [ ]:
"""
We will create a helper function with default keyword arguments to abstact away
Folium's complexity. This leaves us with a simple API we can use going forward.
https://alysivji.github.io/getting-started-with-folium.html
"""
def map_points(
    df,
    lat_col = 'lat',
    lon_col = 'lon',
    zoom_start = 11, 
    plot_points = False,
    pt_radius = 15, 
    draw_heatmap = False,
    heat_map_weights_col = None, 
    heat_map_weights_normalize = False,
    heat_map_radius = 15,
    popup_name = 'nombre'
):
  """
  Creates a map given a dataframe of points. 
  Can also produce a heatmap overlay

  Arg:
    df: dataframe containing points to maps
    lat_col: Column containing latitude (string)
    lon_col: Column containing lonitude (string)
    zoom_start: Integer representing the initial zoom of the map
    plot_points: Add points to map (boolean)
    pt_radius: Size of each point
    draw_heatmap: Add heatmap to map (boolean)
    heat_map_weights_col: Column containing heatmap weights
    heat_map_weights_normalize: Normalize heatmap weights (boolean)
    heat_map_radius: Size of heatmap point
    popup_name: name of the popup

  Returns:
      folium map object
  """

  ## center map in the middle of points center in
  middle_lat = df[lat_col].median()
  middle_lon = df[lon_col].median()

  curr_map = folium.Map(
      location=[middle_lat, middle_lon],
      zoom_start=zoom_start
  )
  cols_to_pull = [ lat_col, lon_col, heat_map_weights_col ]
  # add points to map
  if plot_points:
    for _, row in df.iterrows():
      folium.CircleMarker(
        [ row[lat_col], row[lon_col] ],
        radius = pt_radius,
        popup = row[ popup_name ],
        fill_color = "#3db7e4", # divvy color
      ).add_to( curr_map )
  

  # add heatmap
  if draw_heatmap:
    # convert to (n, 2) or (n, 3) matrix format
    if heat_map_weights_col is None:
      cols_to_pull = [lat_col, lon_col]
    else:
      # if we have to normalize
      if heat_map_weights_normalize:
        df[ heat_map_weights_col] = df[heat_map_weights_col] / df[heat_map_weights_col].sum()
        cols_to_pull = [ lat_col, lon_col, heat_map_weights_col ]
      stations = df[cols_to_pull].values
      curr_map.add_children(
          plugins.HeatMap(
              stations,
              radius = heat_map_radius
          )
      )
  return curr_map

In [ ]:
# del m
m = map_points(
    derivados["estaciones"]["df"],
    plot_points = False,
    draw_heatmap = True,
    heat_map_weights_col = 'salidas',
    heat_map_weights_normalize = True,
)
m

In [ ]:
derivados["recorridos"]["df"].info()

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# Fetch weather from drive
id = '14BpBfjQ3IrfeK8CajCWrEdfPg7S0eqc8'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('weather-data-ba.csv') 
weather_data_df = pd.read_csv('weather-data-ba.csv')
weather_data_df.head()
weather_data_df = weather_data_df.filter(
    ['DATE','PRCP','TAVG']
)
weather_data_df.isnull().sum( axis = 0 )
weather_data_df['PRCP'] = weather_data_df['PRCP'].replace( np.nan, 0 )
weather_data_df.head()

In [ ]:
bicicletas["recorridos_clima_df"].to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos-clima-2020-2021.csv'
)

In [ ]:
bicicletas["recorridos_clima_df"].head()

## Bajar el ultimo
Bajar el dataset limpio desde drive

In [ ]:
from google.colab import files
files.download('recorridos_2020-2021.zip')

In [ ]:
bicicletas["recorridos_df"].to_csv(
    '/drive/My Drive/Datasets-bicing/recorridos_2020-2021.csv'
)

In [ ]:
id = '17liAPD1lyj2wT-Hcbh4ed9-Cn4peoyIq'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('recorridos_2020-2021.csv') 
bicicletas["recorridos_df"] = pd.read_csv('recorridos_2020-2021.csv')
bicicletas["recorridos_df"].head()